# 1.閒聊問答實作
請匯入fuzzy套件，並嘗試判斷與下列語句的相似度:  
`['最近有什麼好看的電影','推薦不錯的電影','那些影片評價不錯', "高評價的電影", "很棒的影片"]`  
<br>
若相似分數 >= 70，則回覆:  
`"最近當然要看鬼滅劇場版阿!"`

In [1]:
from fuzzychinese import FuzzyChineseMatch

In [2]:
#建立中文模糊比對
def fcm_fit(sentence_list : list, analyzer = 'stroke'):    
    fcm = FuzzyChineseMatch(ngram_range = (3, 3), analyzer = analyzer)
    fcm.fit(sentence_list)
    return fcm

def fcm_mapping(query : str, fcm, threshold) -> bool:
    _similar = fcm.transform([query], n=1)
    most_similar_score = fcm.get_similarity_score().max()
    return most_similar_score >= threshold

In [3]:
# chatbot
class chatbotv1():
    def __iniit__(self):
        self.fcm = None
        
    def fit(self, sentence_list):
        self.fcm = fcm_fit(sentence_list)
        
    def reply(self, sentence):
        assert type(sentence) == str
        return '最近當然要看鬼滅劇場版阿!' if fcm_mapping(sentence, self.fcm, 0.7) else '我還沒辦法回應這件事情'

In [4]:
# 推薦我一部電影
chatbot = chatbotv1()

# 建立自定義字句的模糊比對
sentence_list = ['最近有什麼好看的電影', '推薦不錯的電影', '哪些影片評價不錯', '高評價的電影', '很棒的影片']
chatbot.fit(sentence_list)


query = ['推薦我一部電影', '電影', '電影推薦', '影片推薦', '好看的電影', '好看電影', '好看的影片', '很棒電影']
for sentence in query:
    print('Q :',sentence)
    print('A :',chatbot.reply(sentence),'\n')

Q : 推薦我一部電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 電影推薦
A : 最近當然要看鬼滅劇場版阿! 

Q : 影片推薦
A : 最近當然要看鬼滅劇場版阿! 

Q : 好看的電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 好看電影
A : 我還沒辦法回應這件事情 

Q : 好看的影片
A : 我還沒辦法回應這件事情 

Q : 很棒電影
A : 最近當然要看鬼滅劇場版阿! 



# 2.知識問答實作

請嘗試使用 regular expression 取得 html 的右側資訊摘要 ![](president.png)  
並將其作為人物百科的回覆，已解決目前 parsing 結果的不足 ![](error.png)

In [5]:
import requests, urllib, re
from bs4 import BeautifulSoup as bs

def get_google_reply(kw):
    
    # 爬蟲用參數
    headers = {'Referer':'https://accounts.pixiv.net/loginlang=zh&source=pc&view_type=page&ref=wwwtop_accounts_index',
           'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'}
    kw = urllib.parse.quote(kw)
    url = 'https://www.google.com/search?hl=zh-TW&q=' + kw
    
    # 取得 html
    soup = bs(requests.get(url, headers = headers).text)
        
    # 嘗試取得說明內容
    try:
        soup = soup.find('h3',string = r'說明').find_parent('div').find_all('span')
        for context in soup:
            if '。' in str(context):
                return context.string
    
    #沒有說明內容回傳空值
    except:
        return None

In [6]:
query = ['蔡英文', '馬英九', '吳宗憲', '拜登', 'Joe Biden', 'president', 'test']
for sentence in query:
    print('Q :',sentence)
    print('A :',get_google_reply(sentence),'\n')

Q : 蔡英文
A : 蔡英文，中華民國民主進步黨籍政治人物，現任中華民國總統、民主進步黨主席及中華文化總會會長，亦為法律學者與律師，曾擔任過行政院副院長、立法委員、大陸委員會主任委員、國家安全會議諮詢委員等職務。生於臺北市中山區，本籍屏東縣枋山鄉，是客家人和臺灣原住民族排灣族之後裔。 

Q : 馬英九
A : 馬英九，中華民國政治人物、學者，曾任第12、13任中華民國總統、第4、6任中國國民黨主席等職。祖籍江西省永新縣，籍貫湖南省湘潭縣，馬鶴凌和秦厚修之子，生於英屬香港九龍油麻地，1952年隨雙親定居臺灣臺北市。
畢業於國立臺灣大學法律學系，其後前往美國深造，獲紐約大學法學碩士學位、哈佛大學司法學博士學位。 

Q : 吳宗憲
A : 吳宗憲，台灣男藝人、主持人、歌手、唱片製作人、演員、商人，生於臺灣臺南市中西區。因參與電視歌唱比賽節目《五燈獎》而嶄露頭角，1987年以歌手身分出道，早年有「星馬王子」之名。 

Q : 拜登
A : 小約瑟夫·羅比內特·拜登，通稱喬·拜登，美國民主黨籍政治人物，現任美國總統。此前曾於2009年至2017年擔任第47任美國副總統；於1973年至2009年間擔任家鄉德拉瓦州的聯邦參議員，是美國政壇資深政治人物。
拜登曾在1988年和2008年兩次競選過民主黨的總統候選人提名，但都沒有成功。 

Q : Joe Biden
A : 小約瑟夫·羅比內特·拜登，通稱喬·拜登，美國民主黨籍政治人物，現任美國總統。此前曾於2009年至2017年擔任第47任美國副總統；於1973年至2009年間擔任家鄉德拉瓦州的聯邦參議員，是美國政壇資深政治人物。
拜登曾在1988年和2008年兩次競選過民主黨的總統候選人提名，但都沒有成功。 

Q : president
A : 蔡英文，中華民國民主進步黨籍政治人物，現任中華民國總統、民主進步黨主席及中華文化總會會長，亦為法律學者與律師，曾擔任過行政院副院長、立法委員、大陸委員會主任委員、國家安全會議諮詢委員等職務。生於臺北市中山區，本籍屏東縣枋山鄉，是客家人和臺灣原住民族排灣族之後裔。 

Q : test
A : None 



# 3.聊天機器人answer selection  
請嘗試加入"形容詞"及"副詞"的判斷，以優化"閒聊"與"知識"問答的分類效果

In [7]:
# 青雲客 API
def get_api_ans(query):
    URL = f"http://api.qingyunke.com/api.php?key=free&appid=0&msg={query}"
    headers = {'Content-Type': 'application/json'}
    res = requests.get(URL, headers=headers)

    return res.json()['content']

In [8]:
# jieba斷詞並計算詞性數量
import jieba
import jieba.posseg
jieba.setLogLevel(jieba.logging.INFO)
jieba.set_dictionary('./dict.txt.big')

def pos_counter(query):
    word_pos = set(jieba.posseg.cut(query))

    count_dict = {k:0 for k in ['count_names', 'count_nouns', 'count_verbs', 'count_advs', 'count_adjs']}
    pos_dict ={'names':['nr', 'nz', 'nrt', 'nrfg'],
               'nouns':['n', 'ng', 'ns', 'nt'],
               'verbs':['v', 'vg', 'vd', 'vi', 'vn', 'vq'],
               'advs':['a', 'ag', 'ad', 'an', 'b'],
               'adjs':['d', 'dg', 'df', 'e', 'zg']}

    for word, pos in word_pos:
        for pos_key, pos_list in pos_dict.items():
            if pos in pos_list:
                count_dict['count_' + pos_key] += 1
                break

    return (len(word_pos), *count_dict.values())

In [9]:
# 依詞性數量分類問題
def query_classifier(query):
    token_len, count_names, count_nouns, count_verbs, count_advs, count_adjs = pos_counter(query)
    
    # 比對出專有名詞 不應忽略
    if count_names > 0:
        return False
    # 若完全沒比對出名詞 可能為口語短句 應忽略
    elif count_nouns == 0:
        return True
    # 若比對出副詞但token長度 < 3 可能為口語短句 應忽略
    elif count_advs != 0 and token_len <= 3:
        return True
    # 若一個名詞 + 一個形容詞或副詞 判斷其僅為名詞的補充說明 應忽略
    elif count_nouns == 1 and (count_advs != 0 or count_adjs != 0):
        if token_len < 4:
            return True
        else:
            return False
    # 若動詞超過兩個 可能為非口語短句 不應忽略
    elif count_verbs >= 2:
        return False
    else:
        return False

In [10]:
# 優化聊天機器人判斷流程
class chatbotv2(chatbotv1):
    def __iniit__(self):
        super().__init__()
        
    def reply(self, query):
        res = query_classifier(query)
    
        # 先判斷是否為自定義語句
        if fcm_mapping(sentence, self.fcm, 0.7):
                return '最近當然要看鬼滅劇場版阿!'
        else:
            # 閒聊問句
            if res:
                return get_api_ans(query)
            # 知識問句
            else:
                return get_google_reply(query)   

In [11]:
chatbot = chatbotv2()

# 建立自定義字句的模糊比對
sentence_list = ['最近有什麼好看的電影', '推薦不錯的電影', '哪些影片評價不錯', '高評價的電影', '很棒的影片']
chatbot.fit(sentence_list)
query = ['推薦我一部電影', '電影', '電影推薦', '影片推薦', '好看的電影', '好看電影', '好看的影片', '很棒電影',
         '蔡英文', '馬英九', '吳宗憲', '拜登']

for sentence in query:
    print('Q :',sentence)
    print('A :',chatbot.reply(sentence),'\n')

Q : 推薦我一部電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 電影推薦
A : 最近當然要看鬼滅劇場版阿! 

Q : 影片推薦
A : 最近當然要看鬼滅劇場版阿! 

Q : 好看的電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 好看電影
A : 是啊，很好看 

Q : 好看的影片
A : None 

Q : 很棒電影
A : 最近當然要看鬼滅劇場版阿! 

Q : 蔡英文
A : 蔡英文，中華民國民主進步黨籍政治人物，現任中華民國總統、民主進步黨主席及中華文化總會會長，亦為法律學者與律師，曾擔任過行政院副院長、立法委員、大陸委員會主任委員、國家安全會議諮詢委員等職務。生於臺北市中山區，本籍屏東縣枋山鄉，是客家人和臺灣原住民族排灣族之後裔。 

Q : 馬英九
A : {face:81}太深奥了吧,我真心没弄明白您的话 

Q : 吳宗憲
A : 虽然我很聪明，但也不可能全部都懂哦 

Q : 拜登
A : 小約瑟夫·羅比內特·拜登，通稱喬·拜登，美國民主黨籍政治人物，現任美國總統。此前曾於2009年至2017年擔任第47任美國副總統；於1973年至2009年間擔任家鄉德拉瓦州的聯邦參議員，是美國政壇資深政治人物。
拜登曾在1988年和2008年兩次競選過民主黨的總統候選人提名，但都沒有成功。 

